## WikiGolf

Goal: To reach a target page in as few links as possible

Description:
WikiGolf is a program intended to browse Wikipedia with some degree of intelligence. It will accept two parameters, a starting page and a target. It will begin at the starting page and check the links for anything related to the target, and if no relations are found it will choose one at random. It will then examine that page and check any of its links for pages related to the target, and so forth until it reaches a page that is within the threshold for relevence to the target. 

In [2]:
import numpy as np
import requests
import wikipedia
import gensim
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import wikisearch
import nltk
from nltk.corpus import stopwords
import nltk.collocations
from nltk import FreqDist, word_tokenize
import string
import re

In [16]:
S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

TITLE = 'Jurassic Park (novel)'

PARAMS = {
    'action': "query",
    'titles': TITLE,
    'prop': "links",
    'pllimit': "max",
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
data = R.json()
#print(data['query']['title'])

In [29]:
data['query']['pages'][list(data['query']['pages'])[0]]['links'][:20]

[{'ns': 0, 'title': '2000 AD (comics)'},
 {'ns': 0, 'title': 'A Case of Need'},
 {'ns': 0, 'title': 'Airframe (novel)'},
 {'ns': 0, 'title': 'Alfred A. Knopf'},
 {'ns': 0, 'title': 'Amazon (video game)'},
 {'ns': 0, 'title': 'Amber'},
 {'ns': 0, 'title': 'Amphibian'},
 {'ns': 0, 'title': 'Amusement park'},
 {'ns': 0, 'title': 'Ancient DNA'},
 {'ns': 0, 'title': 'Andrew Ferguson'},
 {'ns': 0, 'title': 'Auxotrophy'},
 {'ns': 0, 'title': 'BILBY Award'},
 {'ns': 0, 'title': 'Backdoor (computing)'},
 {'ns': 0, 'title': 'Barnes & Noble'},
 {'ns': 0, 'title': 'Battle at Big Rock'},
 {'ns': 0, 'title': 'Beyond Westworld'},
 {'ns': 0, 'title': 'Binary (novel)'},
 {'ns': 0, 'title': 'Biotechnology'},
 {'ns': 0, 'title': 'Bird'},
 {'ns': 0, 'title': 'Canopy Flyer'}]

In [30]:
links = data['query']['pages'][list(data['query']['pages'])[0]]['links']

In [38]:
n = np.random.randint(0,len(links))
links[n]

{'ns': 0, 'title': 'DNA'}

In [46]:
n = np.random.randint(0,len(links))
NEXT = links[n]['title']

PARAMS = {
    'action': "query",
    'titles': NEXT,
    'prop': "links",
    'pllimit': "max",
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
next_data = R.json()
print(links[n]['title'])
print(next_data['query']['pages'][list(next_data['query']['pages'])[0]]['title'])

Jurassic World: Original Motion Picture Soundtrack
Jurassic World: Original Motion Picture Soundtrack


In [48]:
def get_links(TITLE):
    PARAMS = {
        'action': "query",
        'titles': TITLE,
        'prop': "links",
        'pllimit': "max",
        'format': "json",
    }

    R = S.get(url=URL, params=PARAMS)
    return R.json()

In [52]:
start = input()
hops = input()

print("0 : " + start)
title = start
for i in range(int(hops)):
    response = get_links(title)
    links = response['query']['pages'][list(response['query']['pages'])[0]]['links']
    n = np.random.randint(0,len(links))
    title = links[n]['title']
    print(str(i+1) + " : " + title)

 Jurassic Park (novel)
 5


0 : Jurassic Park (novel)
1 : Dragon curve
2 : Blancmange curve
3 : Archimedes
4 : Menander
5 : Akrai


In [54]:
jp = wikipedia.page('Jurassic Park (novel)')

Determine threshold for topic

In [102]:
get_50_most_common(wikipedia.page("Batman: No Man's Land"))[:10]

[('gotham', 39),
 ('city', 31),
 ('batman', 27),
 ('land', 22),
 ("man's", 20),
 ('gordon', 17),
 ('story', 11),
 ('two', 11),
 ('comics', 10),
 ('joker', 9)]

In [76]:
get_50_most_common(wikipedia.page('Catwoman'))[:10]

[('batman', 179),
 ('catwoman', 155),
 ('selina', 142),
 ('bruce', 50),
 ('two', 32),
 ('vol', 30),
 ('series', 29),
 ('story', 29),
 ('kyle', 28),
 ('one', 28)]

In [79]:
get_50_most_common(wikipedia.page('Bread'))[:20]

[('bread', 100),
 ('dough', 39),
 ('flour', 30),
 ('yeast', 25),
 ('leavening', 20),
 ('water', 18),
 ('breads', 18),
 ('used', 17),
 ('gluten', 14),
 ('wheat', 13),
 ('baking', 12),
 ('time', 12),
 ('made', 11),
 ('may', 11),
 ('protein', 11),
 ('process', 10),
 ('sourdough', 9),
 ('use', 9),
 ('also', 9),
 ('baked', 9)]

In [80]:
get_50_most_common(wikipedia.page('Cake'))[:10]

[('cake', 75),
 ('cakes', 60),
 ('flour', 19),
 ('made', 14),
 ('sugar', 13),
 ('butter', 13),
 ('baking', 13),
 ('bread', 12),
 ('sponge', 12),
 ('baked', 11)]

In [40]:
wikisearch.normalized_top_50(wikipedia.page("Spider-man").content)[:20]

[('spider', '0.06055'),
 ('parker', '0.01435'),
 ('comic', '0.01396'),
 ('peter', '0.01376'),
 ('charact', '0.01121'),
 ('amaz', '0.009632'),
 ('marvel', '0.008846'),
 ('issu', '0.008649'),
 ('superhero', '0.007273'),
 ('stori', '0.007273'),
 ('book', '0.007077'),
 ('seri', '0.006487'),
 ('origin', '0.005308'),
 ('time', '0.005308'),
 ('ditko', '0.005111'),
 ('also', '0.004914'),
 ('becom', '0.004718'),
 ('power', '0.004521'),
 ('best', '0.004325'),
 ('first', '0.004128')]

In [115]:
l = wikipedia.page('Batman').links

In [123]:
l[:5]

['"Holy..."',
 '/Film',
 '100 (DC Comics)',
 '1939 in comics',
 '711 (Quality Comics)']

In [24]:
def check_links(current, target, visited):

    #combine into list
    page_text = current.content.split(".") + target.content.split(".")

    text = []

     #format for word2vec
    for clue in page_text:
        sentence = clue.translate(str.maketrans('','',string.punctuation)).split(' ')
        new_sent = [word.lower() for word in sentence]   
        text.append(new_sent)
    
    #create model
    model = gensim.models.Word2Vec(text,sg=1)
    model.train(text, total_examples=model.corpus_count, epochs=model.epochs)
    
    #get links from current
    #check links against model for relevence to target subject
    links = current.links
    success = []
    errors = []
    for l in links:
        for word in l.split(' '):
            word = word.lower()
            try:
                dist = model.wv.distance(word,"batman")
                if dist < 0.008 and l not in visited:
                    success.append((l,dist))
                    break
            except KeyError:
                errors.append(word)
    
    if len(success) > 0:
        success.sort(key=lambda tup: tup[0])
        return success[0][0]
    else:
        skiplist = ['Wikipedia', 'Category']
        title = links[np.random.randint(0,len(links))]
        while any(sub in title for sub in skiplist):
            #print(title)
            title = links[np.random.randint(0,len(links))]
        return title

In [46]:
start = 'Jurassic Park (novel)'#input()
target = 'Batman'#input()
target_page = wikipedia.page(target)
print("0 : " + wikipedia.page(start).title)
path = [start]
visited = set(start)
title = start
exit = False
for i in range(200):
    try:
        page = wikipedia.page(title)
        top_20 = wikisearch.get_50_most_common(page.content)[:20]
        for word in top_20:
            if word[0] == target.lower():
                exit = True
        if exit:
            break
        search_success = True
    except wikipedia.exceptions.DisambiguationError:
        search_success = False
    except wikipedia.exceptions.PageError:
        search_success = False
        
    title = check_links(page,target_page,visited)
    visited.add(title)
    path.append(title)
    if i % 100 == 0:
        print(".",end="")

print("Page Found!") if exit else print("Not Found")
print(str(i) + " Hop(s), Finish at : " + title)

0 : Jurassic Park (novel)
..Not Found
199 Hop(s), Finish at : Chambers of parliament


In [45]:
path[:100]

['A Case of Need',
 'Abortion in the United States',
 'Labor unions in the United States',
 'Trade unions in Cuba',
 '1947–1948 Civil War in Mandatory Palestine',
 'Operation Blacklist Forty',
 '1947–1949 Palestine war',
 'Ndogboyosoi War',
 '1994 Bophuthatswana crisis',
 "1946 African Mine Workers' Union strike",
 '1948 South African general election',
 '1910 Cape of Good Hope Provincial Council election',
 '1910 South African general election',
 '1989 South African general election',
 '1915 South African general election',
 '1917 Cape of Good Hope Provincial Council election',
 '1920 South African general election',
 '1921 South African general election',
 '1921 in South Africa',
 '1901 in South Africa',
 '1898 in South Africa',
 '1895 in South Africa',
 '1892 in South Africa',
 '1889 in South Africa',
 '1886 in South Africa',
 '1883 in South Africa',
 '1880 in South Africa',
 '1877 in South Africa',
 '1874 in South Africa',
 '1871 in South Africa',
 '1868 in South Africa',
 '1865 in

### Analyzing with Word2Vec

In [23]:

page1 = wikipedia.page('Bread')
page2 = wikipedia.page('Batman')
page_text = page1.content.split(".") + page2.content.split(".")

text = []

for clue in page_text:
    sentence = clue.translate(str.maketrans('','',string.punctuation)).split(' ')
    new_sent = [word.lower() for word in sentence]   
    text.append(new_sent)

In [24]:
text[:2]

[['bread',
  'is',
  'a',
  'staple',
  'food',
  'prepared',
  'from',
  'a',
  'dough',
  'of',
  'flour',
  'and',
  'water',
  'usually',
  'by',
  'baking'],
 ['',
  'throughout',
  'recorded',
  'history',
  'it',
  'has',
  'been',
  'a',
  'prominent',
  'food',
  'in',
  'large',
  'parts',
  'of',
  'the',
  'world',
  'and',
  'is',
  'one',
  'of',
  'the',
  'oldest',
  'manmade',
  'foods',
  'having',
  'been',
  'of',
  'significant',
  'importance',
  'since',
  'the',
  'dawn',
  'of',
  'agriculture']]

In [25]:
model = gensim.models.Word2Vec(text,sg=1)

In [26]:
model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

(36979, 88430)

In [90]:
model.corpus_total_words

13668

In [47]:
model.wv.distance("baked","batman")

0.0086287260055542

In [51]:
links1 = page1.links
success = []
errors = []
for l in links1:
    for word in l.split(' '):
        word = word.lower()
        try:
            if model.wv.distance(word,"batman") < 0.008:
                success.append(l)
                break
        except KeyError:
            errors.append(word)
print(success)

['AWB Limited', 'Alfred A. Knopf', 'Bosnia and Herzegovina cuisine', 'Bread and circuses', 'Bread and salt', 'Bread in Europe', 'Bread in culture', 'Cuisine of São Tomé and Príncipe', 'Cuisine of the Americas', 'Cuisine of the Central African Republic', 'Cuisine of the Indian subcontinent', 'Cuisine of the Mizrahi Jews', 'Cuisine of the Pennsylvania Dutch', 'Cuisine of the Sephardic Jews', 'Cuisine of the Thirteen Colonies', 'Culture of Himachal Pradesh', 'Democratic Republic of the Congo cuisine', 'Diet in Hinduism', 'Diet in Sikhism', 'Early modern European cuisine', 'English cuisine', 'English language', 'Eurasian cuisine of Singapore and Malaysia', 'Food and Agriculture Organization', 'Gliadin', 'History of Chinese cuisine', 'History of agriculture', 'History of bread', 'History of cuisine from the Indian subcontinent', 'History of seafood', 'History of vegetarianism', 'Indigenous cuisine of the Americas', 'International Standard Book Number', 'List of brand name breads', 'List of 

In [52]:
print(len(success))
print(len(links1))

60
710


In [91]:
model.wv.most_similar("batman")

[('\ntwoface', 0.9981786012649536),
 ('video', 0.997878909111023),
 ('animated', 0.9978392720222473),
 ('universe', 0.9977647066116333),
 ('television', 0.9977205991744995),
 ('movie', 0.9977039098739624),
 ('film', 0.9976729154586792),
 ('liveaction', 0.9976227879524231),
 ('series', 0.997494637966156),
 ('first', 0.9974403977394104)]

### Test with LDA Model

In [3]:
cw_page = wikipedia.page('Catwoman')
cw_links = cw_page.links
cw_links[:10]

['"Holy..."',
 '711 (Quality Comics)',
 'A Death Worse Than Fate',
 'Abin Sur',
 'Ace Magazines (comics)',
 'Ace the Bat-Hound',
 'Action Comics',
 'African-American',
 'Air Wave',
 'Airboy']

In [4]:
words_stopped = [wikisearch.preprocess(cw_page.content)]
words_stopped[0][:10]

['holi']

In [5]:
dictionary = gensim.corpora.Dictionary(words_stopped)
bow_corpus = [dictionary.doc2bow(doc) for doc in words_stopped]

In [6]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [7]:
lda = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [11]:
for idx, topic in lda.print_topics(-1):
    print(type(topic))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [38]:
test_page = wikipedia.page('Baking')
test_words = wikisearch.preprocess(test_page)
tdictionary = gensim.corpora.Dictionary([test_words])
bow_test = [dictionary.doc2bow(test_words)]
test = lda[bow_test]

In [33]:
type(test)

gensim.interfaces.TransformedCorpus

In [39]:
for topic in test:
    print(topic)

[(0, 1.0)]
